In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By # Importar By para localizar elementos
import time # Para adicionar pausas, se necessário
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import sys
from collections import defaultdict
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:

# Funções auxiliares
def wait_for_select_options(driver, select_id, min_options=2, timeout=25):
    WebDriverWait(driver, timeout).until(
        lambda d: len(Select(d.find_element(By.ID, select_id)).options) >= min_options
    )

def wait_for_grade_carregar(driver, timeout=30):
    WebDriverWait(driver, timeout).until(
        EC.visibility_of_element_located((By.ID, "gradeCurricular"))
    )
    WebDriverWait(driver, timeout).until(
        lambda d: len(d.find_elements(By.XPATH, "//div[@id='gradeCurricular']//tr")) > 0
    )

def esperar_overlay_sumir(driver, timeout=30):
    try:
        WebDriverWait(driver, timeout).until(
            EC.invisibility_of_element_located((By.CSS_SELECTOR, "div.blockUI.blockOverlay"))
        )
    except Exception:
        print("Overlay não desapareceu.")

def esperar_overlay_e_clique(driver, elemento, timeout=30):
    try:
        # Aguarda que o overlay desapareça
        WebDriverWait(driver, timeout).until(
            EC.invisibility_of_element_located((By.CSS_SELECTOR, "div.blockUI.blockOverlay"))
        )
        # Move para o elemento e clica
        driver.execute_script("arguments[0].scrollIntoView(true);", elemento)
        WebDriverWait(driver, timeout).until(EC.element_to_be_clickable(elemento)).click()
    except Exception as e:
        print(f"Erro ao tentar clicar no elemento: {e}")

def fechar_popup_erro_se_existir(driver):
    try:
        popup = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located(
                (By.XPATH, "//div[contains(@class, 'ui-dialog') and contains(., 'Dados não encontrados')]")
            )
        )
        botao_fechar = popup.find_element(By.XPATH, ".//span[text()='Fechar']/..")
        botao_fechar.click()
        WebDriverWait(driver, 5).until(EC.staleness_of(popup))
        return True
    except Exception:
        return False

def esperar_botao_buscar_habilitado(driver, timeout=15):
    return WebDriverWait(driver, timeout).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='Buscar']/.."))
    )

def extrair_disciplinas(soup):
    """
    Função para extrair disciplinas obrigatórias e optativas de uma página carregada.
    """
    grade_div = soup.find('div', {'id': 'gradeCurricular'})
    if not grade_div:
        return {"Obrigatórias": [], "Optativas": []}

    structured_data = {"Obrigatórias": [], "Optativas": []}

    tables = grade_div.find_all('table')
    for table in tables:
        table_title_row = table.find('tr', style=lambda x: x and 'rgb(16, 148, 171)' in x)
        if not table_title_row:
            continue
        table_title = table_title_row.get_text(strip=True)
        is_obrigatoria = "Obrigatórias" in table_title

        rows = table.find_all('tr', style=lambda x: x and 'height' in x)
        for row in rows:
            cells = row.find_all('td')
            if len(cells) >= 8:
                discipline_data = {
                    "Código": cells[0].get_text(strip=True),
                    "Nome": cells[1].get_text(strip=True),
                    "Créditos Aula": cells[2].get_text(strip=True),
                    "Créditos Trabalho": cells[3].get_text(strip=True),
                    "CH": cells[4].get_text(strip=True),
                    "CE": cells[5].get_text(strip=True),
                    "CP": cells[6].get_text(strip=True),
                    "ATPA": cells[7].get_text(strip=True),
                }

                if is_obrigatoria:
                    structured_data["Obrigatórias"].append(discipline_data)
                else:
                    structured_data["Optativas"].append(discipline_data)

    return structured_data

def coletar_dados(num_unidades=3):
    driver = webdriver.Chrome()
    URL = "https://uspdigital.usp.br/jupiterweb/jupCarreira.jsp?codmnu=8275"
    driver.get(URL)

    unidades = []

    try:
        wait_for_select_options(driver, "comboUnidade")
        select_unidades = Select(driver.find_element(By.ID, "comboUnidade"))

        for i in range(1, num_unidades + 1):
            select_unidades.select_by_index(i)
            unidade_nome = select_unidades.first_selected_option.text.strip()
            print(f"\nProcessando unidade: {unidade_nome}")
            unidade = Unidade(unidade_nome)

            wait_for_select_options(driver, "comboCurso")
            select_cursos = Select(driver.find_element(By.ID, "comboCurso"))
            cursos_options = [opt for opt in select_cursos.options if opt.get_attribute('value') != '']

            for option in cursos_options:
                try:
                    select_cursos.select_by_value(option.get_attribute('value'))
                    curso_nome = option.text.strip()

                    buscar_btn = esperar_botao_buscar_habilitado(driver)
                    buscar_btn.click()

                    WebDriverWait(driver, 30).until(
                        EC.element_to_be_clickable((By.ID, "step4-tab"))
                    )

                    if fechar_popup_erro_se_existir(driver):
                        continue

                    esperar_overlay_sumir(driver)
                    grade_tab = driver.find_element(By.ID, "step4-tab")
                    esperar_overlay_e_clique(driver, grade_tab)

                    wait_for_grade_carregar(driver)
                    soup = BeautifulSoup(driver.page_source, 'html.parser')
                    disciplinas = extrair_disciplinas(soup)

                    curso = Curso(curso_nome, unidade_nome, "N/A", "N/A", "N/A")
                    curso.disciplinas_obrigatorias.extend(disciplinas["Obrigatórias"])
                    curso.disciplinas_optativas.extend(disciplinas["Optativas"])

                    unidade.adicionar_curso(curso)

                    aba_dados = driver.find_element(By.ID, "step1-tab")
                    esperar_overlay_e_clique(driver, aba_dados)

                except Exception as e:
                    print(f"Erro no curso {option.text}: {e}")
                    aba_dados = driver.find_element(By.ID, "step1-tab")
                    esperar_overlay_e_clique(driver, aba_dados)

            unidades.append(unidade)

    finally:
        driver.quit()

    return unidades


In [5]:
class Disciplina:
    def __init__(self, codigo, nome, creditos):
        self.codigo = codigo
        self.nome = nome
        self.creditos = creditos
class Unidade:
    def __init__(self, nome):
        self.nome = nome
        self.cursos = []
    def adicionar_curso(self, curso):
        self.cursos.append(curso)
class Curso:
    def __init__(self, nome, unidade, turno="N/A", modalidade="N/A", grau="N/A"):
        self.nome = nome
        self.unidade = unidade
        self.turno = turno
        self.modalidade = modalidade
        self.grau = grau
        self.disciplinas_obrigatorias = []
        self.disciplinas_optativas = []

In [ ]:
# Execução
if __name__ == "__main__":
    dados = coletar_dados(num_unidades=3)
    for unidade in dados:
        print(f"\nUnidade: {unidade.nome}")
        for curso in unidade.cursos:
            print(f"  Curso: {curso.nome}")
            print(f"    Disciplinas obrigatórias: {curso.disciplinas_obrigatorias}")



Processando unidade: Escola de Artes, Ciências e Humanidades - ( EACH )

Processando unidade: Escola de Comunicações e Artes - ( ECA )
